In [2]:
import sys
!{sys.executable} -m pip install openCV-python
!{sys.executable} -m pip install imutils

In [3]:
def current_milli_time():
    t = time.time()
    ms = t * 1000
    rms = round(ms)
    ims = int(rms)
    return ims

In [4]:
import numpy as np
import cv2
import math
import time

In [5]:
cap = cv2.VideoCapture(0)

backSub = cv2.createBackgroundSubtractorMOG2()
    
#Add various templates to arrays
templates=[] #Templates for static gestures (left and right hand)
templates.append(cv2.imread("Templates/peace_template_R.jpg", -1))
templates.append(cv2.imread("Templates/peace_template_L.jpg", -1))
templates.append(cv2.imread("Templates/hand_template_R.jpg", -1))
templates.append(cv2.imread("Templates/hand_template_L.jpg", -1))
gesture_templates=[] #Templates for finger wave (various angles)
gesture_templates.append(cv2.imread("Templates/g1.jpg", -1))
gesture_templates.append(cv2.imread("Templates/g2.jpg", -1))
gesture_templates.append(cv2.imread("Templates/g3.jpg", -1))
gesture_templates.append(cv2.imread("Templates/g4.jpg", -1))

#Process each template to return a white/black threshold map to identify hand shape
for i in range(len(templates)):
    template = templates[i]
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.GaussianBlur(template,(5,5),0)
    ret,template = cv2.threshold(template,200,255,cv2.THRESH_BINARY_INV)
    templates[i]=template
for i in range(len(gesture_templates)):
    template = gesture_templates[i]
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.GaussianBlur(template,(5,5),0)
    ret,template = cv2.threshold(template,200,255,cv2.THRESH_BINARY_INV)
    gesture_templates[i]=template

# read first frame
ret, prev_frame = cap.read()

last_track = current_milli_time()
track = []

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_copy = frame.copy()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gauss = cv2.GaussianBlur(gray,(5,5),0)
    ret,thresh2 = cv2.threshold(gauss,150,255,cv2.THRESH_BINARY_INV)
    
    #Detect motion
    fgMask = backSub.apply(frame)
    motion = False
    for i in range(len(gesture_templates)):   #Check for each angle of hand shape
        template = gesture_templates[i]
        w, h = template.shape[::-1]
        tm = cv2.matchTemplate(fgMask, template, cv2.TM_CCOEFF_NORMED)   #Template matching
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(tm)
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        if(max_val > 0.65):
            motion = True  #Used to avoid static template matching if motion is detected
            cv2.rectangle(frame_copy, top_left, bottom_right, (0,0,255), 2)  #Recognize finger shape and track
            t = current_milli_time()
            if (t - last_track > 200):
                last_track = current_milli_time()
                if (len(track) < 5):
                    track = track + [(current_milli_time(), max_loc[1])]
                elif (len(track) == 5):
                    track = track[1:] + [(current_milli_time(), max_loc[1])]
            break
    
    if (len(track) == 5and current_milli_time() - last_track < 3000 and track[-1][0] - track[0][0] > 1500 and abs(track[-1][1] - track[0][1]) > 40):
        cv2.putText(frame_copy, "Waving Finger", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255))
    
    #If no motion, detect static templates
    if (not motion):
        for i in range(len(templates)):
            template = templates[i]
            w, h = template.shape[::-1]
            tm = cv2.matchTemplate(thresh2, template, cv2.TM_CCOEFF_NORMED)   #Template matching
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(tm)
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            if(max_val>0.68 and i<=1):     #Test for peace sign match
                cv2.rectangle(frame_copy, top_left, bottom_right, (0,255,0), 2)
                cv2.putText(frame_copy, "Peace Sign", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0))
                break
            elif(max_val>0.68 and i>=2):   #Test for open hand match
                cv2.rectangle(frame_copy, top_left, bottom_right, (255,0,0), 2)
                cv2.putText(frame_copy, "Hand", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
                break

    # Display the resulting frame
    #cv2.imshow('Mask', fgMask)
    cv2.imshow('Temp', templates[1])
    cv2.imshow('Thresh', thresh2)
    cv2.imshow('Output', frame_copy)
    prev_frame=frame

    #Exit capture
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()